In [1]:
print("hello")

hello


In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [4]:
%pwd

"c:\\Arjun's Work\\Inspectra\\research"

In [5]:
!mkdir test-repo

In [6]:
path = "test-repo/"

repo = Repo.clone_from("https://github.com/Phoenixarjun/Spam-Filter", to_path=path)


In [7]:
loader = GenericLoader.from_filesystem(path, glob="**/*", suffixes=[".py", ".txt", ".md"], parser=LanguageParser(Language.PYTHON, parser_threshold=0.5))

In [8]:
documents = loader.load()

In [9]:
len(documents)

8

In [10]:
documents

[Document(metadata={'source': 'test-repo\\app.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}, page_content='def home():\n    return render_template("index.html", email="", prediction=None)'),
 Document(metadata={'source': 'test-repo\\app.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}, page_content='def predict():\n    email = request.form.get(\'content\')\n    prediction = make_predictions(email)\n    return render_template("index.html", prediction=prediction, email=email)'),
 Document(metadata={'source': 'test-repo\\app.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>}, page_content="def predict_api():\n    data = request.get_json(force=True)\n    email = data['content']\n    prediction = make_predictions(email)\n    return jsonify({'Prediction':prediction, 'Email':email})"),
 Document(metadata={'source': 'test-repo\\app.py', 'content_type': 'simplified_code', 'language': <Language

In [11]:
document_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=500, chunk_overlap=20)

In [12]:
texts = document_splitter.split_documents(documents)
len(texts)

8

In [13]:
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [14]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [15]:
embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001", google_api_key=GOOGLE_API_KEY
    )

In [17]:
vectordb = Chroma.from_documents(
    texts,
    embeddings,
    persist_directory="./db",
)

In [18]:
vectordb.persist()

C:\Users\phoen\AppData\Local\Temp\ipykernel_29572\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [21]:
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY, temperature=0.5)

In [22]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

C:\Users\phoen\AppData\Local\Temp\ipykernel_29572\1764516168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


In [23]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [24]:
question = "What is the purpose of the SpamFilter class?"
result = qa({"question": question})

C:\Users\phoen\AppData\Local\Temp\ipykernel_29572\1580042990.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question": question})


In [25]:
result["answer"]

'There is no mention of a `SpamFilter` class in the provided code. The code implements a spam detection function (`make_predictions`) and integrates it into a Flask web application for both direct user interaction and API access.\n'